In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import app

###1) Data is split into train and test
#   X-ray scans are in grayscale
#   Three classes (Covid, Normal, Pneumonia), so this is a multi-class classification problem

###2) Initialise ImageDataGenerator object
data_generator = ImageDataGenerator(rescale = 1.0/255)
# print its attributes:
print(data_generator.__dict__)

###3) Loading the data (iterators)
# set batch size 
BATCH_SIZE = 16

# training iterator 
training_iterator = data_generator.flow_from_directory('augmented-data/test', class_mode = 'categorical', color_mode = 'grayscale', batch_size = BATCH_SIZE)

# validation iterator
validation_iterator = data_generator.flow_from_directory('augmented-data/train', class_mode = 'categorical', color_mode = 'grayscale', batch_size = BATCH_SIZE)

# sample batch of data and print its dimensions
sample_batch_input,sample_batch_labels = training_iterator.next()
print(sample_batch_input.shape,sample_batch_labels.shape)

###4) Create classification model
# build model
model = Sequential()

# input shape
model.add(layers.Input(shape = (256, 256, 1)))
# first convolutional layer, 8 filters each 3x3 with strides of 2
model.add(tf.keras.layers.Conv2D(8, 3, strides = 2, activation='relu')) 
# pooling
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# second convolutional layer, 8 filters each 3x3 with strides of 2
model.add(tf.keras.layers.Conv2D(8, 3, strides = 2, activation='relu')) 
# pooling
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# flatten layer
model.add(tf.keras.layers.Flatten())
# hidden dense layer with 16 units
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
# output dense layer, three classes
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

# compile model
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

###5) Fit and evaluate model
# fit model
history = model.fit(training_iterator, steps_per_epoch = training_iterator.samples/BATCH_SIZE,
epochs = 5, validation_data = validation_iterator, validation_steps = validation_iterator.samples/BATCH_SIZE)


# Matplotlib extension below
# plotting categorical and validation accuracy over epochs
fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
ax1.plot(history.history['categorical_accuracy'])
ax1.plot(history.history['val_categorical_accuracy'])
ax1.set_title('model accuracy')
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')
ax1.legend(['train', 'validation'], loc='upper left')

# plotting auc and validation auc over epochs
ax2 = fig.add_subplot(2, 1, 2)
ax2.plot(history.history['auc'])
ax2.plot(history.history['val_auc'])
ax2.set_title('model auc')
ax2.set_xlabel('epoch')
ax2.set_ylabel('auc')
ax2.legend(['train', 'validation'], loc='upper left')

# used to keep plots from overlapping
fig.tight_layout()

# use this savefig call at the end of your graph instead of using plt.show()
fig.savefig('static/images/my_plots.png')


###Implementing a Classification Report and a Confusion Matrix
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

test_steps_per_epoch = np.math.ceil(validation_iterator.samples / validation_iterator.batch_size)
predictions = model.predict(validation_iterator, steps = test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis = 1)
true_classes = validation_iterator.classes
class_labels = list(validation_iterator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names = class_labels)
print(report)

cm = confusion_matrix(true_classes, predicted_classes)
print(cm)